# 使用Transformer进行机器翻译 （未完成）

In [1]:
from __future__ import print_function, unicode_literals, division
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [13]:
SOS_token = 0  # Start of Sentence
EOS_token = 1  # End of Sentence

"""
类 Lang 保存当前语言的的word信息
"""


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: 'SOS', 1: 'EOS'}
        self.n_words = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )


def normalizeString(s):
    """
    小写，整齐，移除非字母字符
    :param s: 
    :return: 
    """
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def readLangs(lang1, lang2, reverse=False):

    print('Reading lines...')

    lines = open('data/%s-%s.txt' % (lang1, lang2),
                 encoding='utf-8').read().strip().split('\n')

    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[-1].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print('Read %s sentence pairs' % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes,dtype=torch.long, device=device).view(-1,1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11484 sentence pairs
Counting words...
Counted words:
fra 4594
eng 2978
['il plaisante toujours .', 'he is always joking .']


In [44]:
inputEmbedding = nn.Embedding(input_lang.n_words, 512)
targetEmbedding = nn.Embedding(output_lang.n_words, 512)

n_epochs = 100

model = nn.Transformer()



optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_func = nn.NLLLoss()

for epoch in range(n_epochs):
    
    input_tensor, output_tensor = tensorsFromPair(random.choice(pairs))
    input_tensor = inputEmbedding(input_tensor)
    target_tensor = inputEmbedding(output_tensor)
#     target_tensor = torch.zeros((output_tensor.size(0),1,output_lang.n_words))

    optimizer.zero_grad()
    out = model(input_tensor, target_tensor)
    loss = loss_func(out, output_tensor)
    #loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    print(loss.item())

ValueError: Expected target size (9, 512), got torch.Size([9, 1])

In [54]:
with torch.no_grad():
    loss_func = nn.NLLLoss()
    input_tensor, output_tensor = tensorsFromPair(random.choice(pairs))
    input_tensor = inputEmbedding(input_tensor)
    target_tensor = torch.zeros((output_tensor.size(0),1,output_lang.n_words))
    for row in range(target_tensor.size(0)):
        target_tensor[row, 0, output_tensor[row, 0]] = 1
    out = model(input_tensor, target_tensor)
    print(out.size(),output_tensor.size())
    print(output_tensor.squeeze(1))
    loss = loss_func(out.squeeze(1), output_tensor.squeeze(1))
    print(loss)

RuntimeError: the feature number of src and tgt must be equal to d_model